## **Gurobi** - Cómo verificar que esté todo instalado y que mi licencia esté funcionando (futuro proximo...)

In [1]:
import gurobipy as gp

try:
    model = gp.Model("test_model")
    print("Gurobi está instalado correctamente.")
except gp.GurobiError as e:
    print(f"Error al intentar crear el modelo: {e}")
except Exception as e:
    print(f"Error general: {e}")


In [2]:
import gurobipy as gp

try:
    gp.Model()  
    print("La licencia de Gurobi es válida.")
except gp.GurobiError as e:
    if e.errno == gp.GRB_ERROR_NO_LICENSE:
        print("La licencia de Gurobi ha caducado o no es válida.")
    else:
        print(f"Ocurrió un error relacionado con la licencia: {e}")


# **Laboratorio 4: Modelos PL en Python con Gurobi**
## Introducción

In [3]:
# PUNTO 1: DEFINICION DEL MODELO

import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB

#Creamos un modelo
modelo = Model("nombre_del_modelo")


In [4]:
# PUNTO 2: DEFINICION DE VARIABLES 
# Ocuparemos 'addVar' para una sola variable o 'addVars' para multiples variables.

#Variables continuas (pueden tomar cualquier valor dentro de un rango especificado)
x = modelo.addVar(lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")

#Variables enteros (solo pueden tomar valores enteros)
y = modelo.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.INTEGER, name="y")

#Variables binarias (solo pueden tomar valores 0 ó 1)
z = modelo.addVar(vtype=GRB.BINARY,name="z")

#Multiples variables (para definir un conjunto de variables indexadas)
I= range(10) #definir conjunto
x_1 = modelo.addVars(I, vtype=GRB.CONTINUOUS, name="x_1")
y_1 = modelo.addVars(I, vtype=GRB.CONTINUOUS, name="y_1")

In [5]:
# PUNTO 3: DEFINICION DE LA F.O. 
#minimizar
modelo.setObjective(3*x + 2*y, GRB.MINIMIZE)

#maximizar
modelo.setObjective(3*x + 2*y, GRB.MAXIMIZE)

In [6]:
# PUNTO 4: DEFINICION DE RESTRICCIONES 

#Restriccion simple
modelo.addConstr(x + y <= 10, "nombre_restriccion")

#Restricciones multiples
modelo.addConstrs((x_1[i]+y_1[i] <= 10 for i in I), "nombre_restriccion2")

#Restriccion con condiciones, utilizando variables binarias
modelo.addConstr( x>= 5 * z, "condicion z")  #si z=1 entonces x>= 5; caso contrario x>=0


In [7]:
# PUNTO 5: PARAMETROS DE OPTIMIZACION

#Tiempo maximo de ejecucion 
modelo.setParam('TimeLimit',100)

#Tolerancia de optimalidad
modelo.setParam('MIPgap',0.01)

#Numero de threads
modelo.setParam('Threads',4)


In [8]:
# PUNTO 6: RESOLUCION DEL MODELO

#Optimizar el modelo
modelo.optimize()

In [9]:
# PUNTO 7: EXTRACCION DE LOS RESULTADOS

#valor de las variables
valor_x=x.x

#Obtener el valor de una F.O
valor_fo=modelo.ObjVal
valor_fo


In [10]:
# PUNTO 8: ANALISIS DE SENSIBILIDAD y OTROS RESULTADOS

#Reduccion de costos
#rc = x.RC

#Sombra de precios
#sp = modelo.getConstrByName("nombre_restriccion").Pi



In [11]:
# PUNTO 9: ESPECIFICACION DE OBJETIVOS MULTIPLES

modelo.setObjectiveN(3*x + 2*y, index=0, priority=1)
modelo.setObjectiveN(2*x + y, index=1, priority = 0)


In [12]:
# PUNTO 10: EXPORTAR e IMPORTAR MODELOS
#gurobi nos opermite exportar un modelo o importarlo

#exportar
modelo.write("model.lp")
modelo.write("model.mps")

#importar o leer
modelo.read("model.lp")


Tarea 2:

In [ ]:
# PUNTO 1: DEFINICION DEL MODELO

import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB

# Crear el modelo
modelo = gp.Model("seleccion_guerreros")

In [ ]:

# Datos del problema
guerreros = {
    'Berserker': {'vitalidad': 800, 'ataque_fisico': 850, 'ataque_arcano': 0, 'velocidad_ataque': 4, 'curacion': 500, 'dano_fisico': 63750},
    'Paladin': {'vitalidad': 1000, 'ataque_fisico': 750, 'ataque_arcano': 500, 'velocidad_ataque': 8, 'curacion': 800, 'dano_fisico': 28125},
    'Arquero': {'vitalidad': 500, 'ataque_fisico': 700, 'ataque_arcano': 300, 'velocidad_ataque': 3, 'curacion': 300, 'dano_fisico': 70000},
    'Mago': {'vitalidad': 400, 'ataque_fisico': 0, 'ataque_arcano': 1000, 'velocidad_ataque': 6, 'curacion': 500, 'dano_fisico': 0},
    'Sacerdote': {'vitalidad': 600, 'ataque_fisico': 0, 'ataque_arcano': 800, 'velocidad_ataque': 8, 'curacion': 1000, 'dano_fisico': 0},
    'Druida': {'vitalidad': 700, 'ataque_fisico': 400, 'ataque_arcano': 700, 'velocidad_ataque': 5, 'curacion': 800, 'dano_fisico': 24000},
    'Elfo': {'vitalidad': 650, 'ataque_fisico': 200, 'ataque_arcano': 900, 'velocidad_ataque': 4, 'curacion': 800, 'dano_fisico': 15000},
    'Asesino': {'vitalidad': 600, 'ataque_fisico': 1000, 'ataque_arcano': 0, 'velocidad_ataque': 1, 'curacion': 200, 'dano_fisico': 300000},
    'Espadachin': {'vitalidad': 700, 'ataque_fisico': 700, 'ataque_arcano': 0, 'velocidad_ataque': 1, 'curacion': 600, 'dano_fisico': 210000},
    'Gigante': {'vitalidad': 1000, 'ataque_fisico': 10000, 'ataque_arcano': 200, 'velocidad_ataque': 10, 'curacion': 400, 'dano_fisico': 300000},
}

In [ ]:

# Variables de decisión
x = modelo.addVars(guerreros.keys(), vtype=GRB.BINARY, name="x")

In [ ]:
# Función objetivo: maximizar el daño físico total
modelo.setObjective(gp.quicksum(guerreros[g]['dano_fisico'] * x[g] for g in guerreros), GRB.MAXIMIZE)

In [ ]:
# Restricción 1: Ataque arcano promedio ≥ 450
modelo.addConstr(gp.quicksum(guerreros[g]['ataque_arcano'] * x[g] for g in guerreros) / gp.quicksum(x[g] for g in guerreros) >= 450, "arcano_promedio")

# Restricción 2: Vitalidad promedio ≥ 500
modelo.addConstr(gp.quicksum(guerreros[g]['vitalidad'] * x[g] for g in guerreros) / gp.quicksum(x[g] for g in guerreros) >= 500, "vitalidad_promedio")

# Restricción 3: Curación promedio ≥ 300
modelo.addConstr(gp.quicksum(guerreros[g]['curacion'] * x[g] for g in guerreros) / gp.quicksum(x[g] for g in guerreros) >= 300, "curacion_promedio")

# Restricción 4: No puede haber un Berserker y un Paladín juntos
modelo.addConstr(x['Berserker'] + x['Paladin'] <= 1, "berserker_paladin")

# Restricción 5: No puede haber un Asesino y un Sacerdote juntos
modelo.addConstr(x['Asesino'] + x['Sacerdote'] <= 1, "asesino_sacerdote")

# Restricción 6: Seleccionar exactamente 4 guerreros
modelo.addConstr(gp.quicksum(x[g] for g in guerreros) == 4, "cuatro_guerreros")
# Optimizar el modelo
modelo.optimize()

In [ ]:

# Resultados
seleccionados = [g for g in guerreros if x[g].x > 0.5]
print(f'Guerreros seleccionados: {seleccionados}')
print(f'Daño físico total: {modelo.objVal}')